1. Preparar datos para enviarlos a la base de datos. 
    - Separar datos agrupados (habrá que asignarles ID?)
    - Exportarlos en csv para leerlos en postgreSQL

# Conexión a BD
## Actividad 5 
### Actividad 5
    1. Deberán generar una instancia de conexión a Base de datos preferiblemente PostgreSQL donde deberá existir el dataset elegido para el proyecto.
    2. Se deben realizar consultas SQL desde el lenguaje Python utilizando las librerías (psycopg2 o sqlalchemy) y quedar evidenciadas en el archivo .ipynb que se entregue

# Desarrollo

## Planificación
Se planea crear una tabla por cada columna. 
### 1. Preparación de los datos
    Se encontraron dos problemáticas con respecto a subir el csv directamente a la base de datos: 
        A. Existen datos que están agrupados en una misma fila. 
        B. Existen datos que serían mejor en formatos distintos a Texto (fecha).
    Debido a esto, se planea hacer lo siguiente: 
        A. Separar los datos agrupados por valores únicos para poder construir una tabla caracteristica de ellos. 
        B. Procesar los datos de Texto a Fecha o Número para un mejor registro. 
        C. Tratar de categorizar los datos categorizables. 
### 2. Creación de la instancia de base de datos. (en preparación)
### 3. Consultas a la base de datos. (en preparación)

## Desarrollo técnico. 
### 1. Preparación de los datos

In [24]:
#Inicialización
import pandas as pd
#import matplotlib as m
#import matplotlib.pyplot as plt
#import plotly.graph_objects as go
#import seaborn as sns
#import plotly as pl
import psycopg2 as psy
import sqlalchemy as sql

#Lectura del archivo y confirmación de contenido

tabla=pd.read_csv('anime.csv',sep=',',encoding='utf-8')
tabla.head(5)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


Despues de un análisis de comportamiento de los datos columna a columna, se llegó a la conclusión de que se van a procesar las siguientes columnas: 

    - Deconstruir en valores únicos: 
        Licensors
        Producers
        Genres

    - Procesar: 
        Aired -> a fecha
        Premiered -> a fecha
        Duration -> a float en minutos


Tablas planificadas: 
- animeBase (archivo base)
- Anime: *Tabla de caracterización de los animes*
    - id (int)
    - name (varchar)
    - score (float)
    - english name (varchar)
    - japanese name (varchar)
    - episodes (int)
    - aired (date)
    - premiered (date)
    - typeId (int)
    - sourceId (int)
    - durationMinutes (float)
    - rateId (int)
    - ranked (int)
    - popularity (int)
    - members (int)
    - favorites (int)
    - watching (int)
    - completed (int)
    - onHold (int)
    - dropped (int)
    - planToWatch (int)
    - score10 (int)
    - score9 (int)
    - score8 (int)
    - score7 (int)
    - score6 (int)
    - score5 (int)
    - score4 (int)
    - score3 (int)
    - score2 (int)
    - score1 (int)

- Genres: 
    - id (int)
    - genre (valores únicos) (varchar)

- AnimeGenre: *Tabla relación* 
    - id (int)
    - animeId (int)
    - genreId (int)

- Type: 
    - id (int)
    - type (varchar)

- Producers: 
    - id (int)
    - producer (valores únicos) (varchar)

- AnimeProducer: *Tabla relación* 
    - id (int)
    - animeId (int)
    - producerId (int)

- Licensors: 
    - id (int)
    - licensor (valores únicos) (varchar)

- AnimeLicensor: *Tabla relación* 
    - id (int)
    - animeId (int)
    - licensorId (int)

- Studios: 
    - id (int)
    - studio (valores únicos) (varchar)

- AnimeStudio: *Tabla relación* 
    - id (int)
    - animeId (int)
    - studioId (int)

- Source 
    - id (int)
    - source (varchar)

- Rating
    - id (int)
    - rate (varchar)


In [38]:
tabla['Genres'][2608]#.count(',')

'Slice of Life, Adventure, Fantasy, Magic, Game, Martial Arts, Comedy, Mecha, Super Power, Supernatural, School, Drama, Sci-Fi'

In [39]:
max = 0
for i in range(len(tabla)):
    if max < tabla['Genres'][i].count(','):
        max = tabla['Genres'][i].count(',')
max

12

In [17]:
tabla.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [44]:
len(tabla['Type'].unique())
#tabla['Type'][567].count(',')

7

In [25]:
for i in tabla.columns:
    print('En '+i)
    print(tabla[i])

En MAL_ID
0            1
1            5
2            6
3            7
4            8
         ...  
17557    48481
17558    48483
17559    48488
17560    48491
17561    48492
Name: MAL_ID, Length: 17562, dtype: int64
En Name
0                           Cowboy Bebop
1        Cowboy Bebop: Tengoku no Tobira
2                                 Trigun
3                     Witch Hunter Robin
4                         Bouken Ou Beet
                      ...               
17557    Daomu Biji Zhi Qinling Shen Shu
17558                       Mieruko-chan
17559    Higurashi no Naku Koro ni Sotsu
17560        Yama no Susume: Next Summit
17561                      Scarlet Nexus
Name: Name, Length: 17562, dtype: object
En Score
0           8.78
1           8.39
2           8.24
3           7.27
4           6.98
          ...   
17557    Unknown
17558    Unknown
17559    Unknown
17560    Unknown
17561    Unknown
Name: Score, Length: 17562, dtype: object
En Genres
0          Action, Adventure, Comed